ecbo:最適化に基づくサイズ均等クラスタリング\
帰属度Uの最適化においてはシンプレックス法を採用

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#sysはコマンドライン引数
import sys
import csv
np.seterr(divide='ignore', invalid='ignore')
#csvファイルの読み込み、一行目はヘッダーじゃない
df = pd.read_csv("",header=None)
J_list = 0
#dfをnumpyで扱えるnp.ndarray変換
X = df.values
#配列の行数、列数の格納
X_size,dimension = X.shape

#クラスタ数宣言
c=5
#Kはクラスタサイズ(整数部分抽出)
K=X_size//c
#PuLPライブラリをインポートする
import pulp
#max繰り返し回数の宣言
max_iter=100

d=np.zeros((X_size,c))
sum_distance = []
#クラスター番号を格納するリスト
#目的関数を格納するリスト
best_J=10000
best_centroids = np.zeros((c,dimension))
u_value = np.zeros((X_size,c))

In [ ]:
def v_cal(u_value):
    #クラスタ中心の計算
    v=np.zeros((c,dimension))
    for i in range(c):    
        #分母
        numerator=0
        #分子
        denominator=0
        for j in range(X_size):
            numerator+=u_value[j][i]
            denominator+=u_value[j][i]*X[j]
        v[i]=denominator/numerator
    return v


for s in range (max_iter):
    centroids = X[np.random.choice(X_size,c)]
    for epoch in range(max_iter):
        for i in range(X_size):
            #各クラスタ中心との距離の格納
            d[i] = np.sum((centroids - X[i]) ** 2, axis=1)
            #print(distances)

        #d =distances.tolist()

        prob = pulp.LpProblem(name="u_best", sense=pulp.LpMinimize)
        #変数定義
        u=[]

        for i in range(X_size):
            sublist = []
            for j in range(c):
                sublist.append(pulp.LpVariable(name='u{}_{}'.format(i,j), cat='Countinuous', lowBound=0))
            u.append(sublist)

        #目的関数
        obj=0
        for i in range(X_size):
            for j in range(c):
                obj += d[i][j]*u[i][j]
        prob.setObjective(obj)

        #制約条件
        for i in range(X_size):
            prob += pulp.lpSum([u[i][j] for j in range(c)]) == 1 # sum
        for j in range(c):
            prob += pulp.lpSum([u[i][j] for i in range(X_size)]) >= K  # sum
            prob += pulp.lpSum([u[i][j] for i in range(X_size)]) <= K + 1 # sum

        #解く
        prob.solve()
        for i in range(X_size):
            for j in range(c):
                u_value[i][j] = u[i][j].value()
        #クラスター中心の更新
        new_centroids = v_cal(u_value)

        #クラスター番号の格納
        cluster = [0 for i in range(X_size)]
        for i in range(X_size):
            for j in range(c):
                if u_value[i][j]==1:
                    cluster[i] =j
                else:
                    continue
        np_cluster = np.array(cluster)
        
        #----------ここまでUとVの交互最適化-----
        #終了条件
        
        if (np.array_equal(new_centroids, centroids)):
            print("breakします")
            break
        
        #繰り返し準備
        centroids = np.copy(new_centroids)
        
    #目的関数の作成
    J = 0
    for i in range(c):
        J+= np.sum((X[np_cluster==i]-centroids[i])**2)
    #この初期値のときの結果が優れている場合，bestを更新    
    if best_J > J:
        best_J = J
        best_centroids =  np.copy(centroids)
    print(s)
    print("回目のクラスタリング結果↓")
    print(best_J)
    print(best_centroids)
    print("終了")
print(np_cluster)
print(best_J)
print(best_centroids)